In [ ]:
from os import path
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive
drive.mount('/content/gdrive')
path_to_csv = '/content/gdrive/MyDrive/Sentiment (2).csv'
!pip install Keras-Preprocessing
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import load_model
!pip install scikeras
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv(path_to_csv)
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
#print(model.summary())


labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)







Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
291/291 - 33s - loss: 0.8252 - accuracy: 0.6409 - 33s/epoch - 113ms/step
144/144 - 2s - loss: 0.7573 - accuracy: 0.6750 - 2s/epoch - 15ms/step
0.7572540640830994
0.6749672293663025
['loss', 'accuracy']


In [ ]:
model.save('sentiment.h5')

In [ ]:

model= load_model('sentiment.h5')


In [ ]:
sentence=["A lot of good things are happening.We are respected again throughout the world, and that's a great thing.@realDonaldTrump"]
sentence= tokenizer.texts_to_sequences(sentence)
sentence= pad_sequences(sentence,maxlen=28,dtype='int32',value=0)
sentiment= model.predict(sentence,batch_size=1,verbose=2)[0]
print(sentiment)
if sentiment.all()==0:
  print("neutral")
elif sentiment.all() < 0:
  print("negative")
elif sentiment.all() > 0:
  print("positive")
else:
  print("cant be determined")


1/1 - 0s - 401ms/epoch - 401ms/step
[0.6731788  0.15662014 0.1702011 ]
positive


In [33]:

model = KerasRegressor (build_fn=createmodel,verbose=2)
batch_size = [10, 20, 40]
epochs = [1, 2]
param_grid = {'batch_size': batch_size, 'epochs':epochs}
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


744/744 - 71s - loss: 0.8238 - accuracy: 0.6457 - 71s/epoch - 95ms/step
186/186 - 2s - 2s/epoch - 9ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


744/744 - 69s - loss: 0.8193 - accuracy: 0.6490 - 69s/epoch - 93ms/step
186/186 - 2s - 2s/epoch - 9ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/2
186/186 - 26s - loss: 0.8447 - accuracy: 0.6344 - 26s/epoch - 139ms/step
Epoch 2/2
186/186 - 22s - loss: 0.6841 - accuracy: 0.7112 - 22s/epoch - 118ms/step
47/47 - 1s - 861ms/epoch - 18ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/2
186/186 - 26s - loss: 0.8467 - accuracy: 0.6356 - 26s/epoch - 141ms/step
Epoch 2/2
186/186 - 24s - loss: 0.6828 - accuracy: 0.7071 - 24s/epoch - 129ms/step
47/47 - 1s - 883ms/epoch - 19ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/2
233/233 - 31s - loss: 0.8331 - accuracy: 0.6413 - 31s/epoch - 134ms/step
Epoch 2/2
233/233 - 29s - loss: 0.6854 - accuracy: 0.7084 - 29s/epoch - 123ms/step
Best: 0.211641 using {'batch_size': 40, 'epochs': 2}
